## Querying weather data from the PostGreSQL DB
examples pertaining to the monthwise-aggregated nature of the DB data

DB schema:
```
CREATE TABLE imd_data(
	sr SERIAL PRIMARY KEY,
	year1 INT NULL,
	month1 INT NULL,
	data JSONB DEFAULT '{}' NOT NULL,
	geometry GEOMETRY(POINT,4326) NOT NULL
);

CREATE TABLE grid(
	sr SERIAL PRIMARY KEY,
	geometry GEOMETRY(POINT,4326) NOT NULL
);
```

In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
# import geopandas as gpd

In [2]:
creds = json.load(open('dbcreds.json','r'))
engine = create_engine(f"postgresql://{creds['DB_USER']}:{creds['DB_PW']}@{creds['DB_SERVER']}:{creds['DB_PORT']}/{creds['DB_DBNAME']}")

### Use case 1: Get following metrics for one location for a 10 year period  : 
- cumulative monthly rainfall
- monthly average max and min temperatures 
- number of days with no rain
- number of days with max temp above 30
- number of dats with min temp below 10

In [3]:
lat, lon = 28.5, 72.5
start_year = 1961
end_year = 1970
highT = 40
lowT = 15

In [4]:
s1 = f"""select year1, month1, data from imd_data
where year1 between {start_year} and {end_year}
and ST_Y(geometry)={lat}
and ST_X(geometry)={lon}
order by year1, month1
"""
print(s1)

select year1, month1, data from imd_data
where year1 between 1961 and 1970
and ST_Y(geometry)=28.5
and ST_X(geometry)=72.5
order by year1, month1



In [5]:
df1 = pd.read_sql(s1, con=engine)

In [6]:
df1

,year1,month1,data
0,1961,1,"{'1961-01-01': {'rain': 0.0, 'tmax': 21.510000..."
1,1961,2,"{'1961-02-01': {'rain': 11.967379570007324, 't..."
2,1961,3,"{'1961-03-01': {'rain': 0.0, 'tmax': 32.369998..."
3,1961,4,"{'1961-04-01': {'rain': 0.0, 'tmax': 35.520000..."
4,1961,5,"{'1961-05-01': {'rain': 0.0, 'tmax': 41.630001..."
...,...,...,...
115,1970,8,"{'1970-08-01': {'rain': 0.0, 'tmax': 37.400001..."
116,1970,9,"{'1970-09-01': {'rain': 0.0, 'tmax': 33.759998..."
117,1970,10,"{'1970-10-01': {'rain': 0.0, 'tmax': 34.419998..."
118,1970,11,"{'1970-11-01': {'rain': 0.0, 'tmax': 32.479999..."


In [7]:
# preview one data cell:
df1.at[0,'data']

{'1961-01-01': {'rain': 0.0, 'tmax': 21.510000228881836, 'tmin': 7.5},
 '1961-01-02': {'rain': 0.0,
  'tmax': 19.479999542236328,
  'tmin': 6.190000057220459},
 '1961-01-03': {'rain': 5.3971171379089355,
  'tmax': 18.969999313354492,
  'tmin': 6.010000228881836},
 '1961-01-04': {'rain': 0.0,
  'tmax': 16.510000228881836,
  'tmin': 4.260000228881836},
 '1961-01-05': {'rain': 0.0,
  'tmax': 18.670000076293945,
  'tmin': 4.130000114440918},
 '1961-01-06': {'rain': 0.0,
  'tmax': 20.579999923706055,
  'tmin': 4.010000228881836},
 '1961-01-07': {'rain': 0.0,
  'tmax': 21.1299991607666,
  'tmin': 5.119999885559082},
 '1961-01-08': {'rain': 0.0,
  'tmax': 21.90999984741211,
  'tmin': 6.150000095367432},
 '1961-01-09': {'rain': 0.0,
  'tmax': 21.200000762939453,
  'tmin': 5.579999923706055},
 '1961-01-10': {'rain': 0.0,
  'tmax': 21.709999084472656,
  'tmin': 5.369999885559082},
 '1961-01-11': {'rain': 0.0,
  'tmax': 20.65999984741211,
  'tmin': 3.490000009536743},
 '1961-01-12': {'rain': 0.0,

so its a dict with dates as keys

#### since the data is already at 1 row = 1 month, we can directly process each row to get the metrics needed

In [8]:
def process1(data):
    # data is going to be the dict above
    rainArr = []
    tmaxArr = []
    tminArr = []
    
    zeroRainCounter = 0
    highTCounter = 0
    lowTCounter = 0
    
    for date1 in data.keys():
        row = data[date1]
        if row.get('rain',False) is not False:
            rainArr.append(row['rain'])
            if row['rain'] == 0: zeroRainCounter += 1
        
        if row.get('tmax',False) is not False:
            tmaxArr.append(row['tmax'])
            if row['tmax'] >= highT: highTCounter += 1
        
        if row.get('tmin',False) is not False:
            tminArr.append(row['tmin'])
            if row['tmin'] <= lowT: lowTCounter += 1
        # the if conditions are there so that only existing records get counted. and Zero should also get counted, not skipped
    
    cumulativeRain = round(sum(rainArr),2)
    avgTmax = round(sum(tmaxArr)/len(tmaxArr),1)
    avgTmin = round(sum(tminArr)/len(tminArr),1)
    
    return cumulativeRain, avgTmax, avgTmin, zeroRainCounter, highTCounter, lowTCounter

In [9]:
df1[['cumulativeRain','avgTmax','avgTmin', 'days_noRain', f"days_over{highT}", f"days_below{lowT}"]] = df1['data'].apply(lambda x: pd.Series(process1(x)))

# this method is used to derive muliple columns out of one column in dataframes

In [10]:
df1

,year1,month1,data,cumulativeRain,avgTmax,avgTmin,days_noRain,days_over40,days_below15
0,1961,1,"{'1961-01-01': {'rain': 0.0, 'tmax': 21.510000...",5.69,22.8,7.1,29.0,0.0,31.0
1,1961,2,"{'1961-02-01': {'rain': 11.967379570007324, 't...",12.87,22.8,8.4,25.0,0.0,28.0
2,1961,3,"{'1961-03-01': {'rain': 0.0, 'tmax': 32.369998...",0.00,32.9,16.1,31.0,0.0,10.0
3,1961,4,"{'1961-04-01': {'rain': 0.0, 'tmax': 35.520000...",0.67,36.5,20.2,29.0,6.0,0.0
4,1961,5,"{'1961-05-01': {'rain': 0.0, 'tmax': 41.630001...",18.84,40.7,25.5,25.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...
115,1970,8,"{'1970-08-01': {'rain': 0.0, 'tmax': 37.400001...",66.12,34.6,25.0,16.0,0.0,0.0
116,1970,9,"{'1970-09-01': {'rain': 0.0, 'tmax': 33.759998...",92.02,34.5,23.4,21.0,0.0,0.0
117,1970,10,"{'1970-10-01': {'rain': 0.0, 'tmax': 34.419998...",0.00,35.5,18.6,31.0,0.0,2.0
118,1970,11,"{'1970-11-01': {'rain': 0.0, 'tmax': 32.479999...",0.00,30.1,9.7,30.0,0.0,30.0


In [14]:
# to do : visualizing this

## Use case 2: Find the nearest grid points to a given location
do proximity search query

In [11]:
lat1, lon1 = 18.4910, 73.8534
radiusKm = 35

In [12]:
s2 = f"""select sr, ST_Y(geometry) as lon, ST_X(geometry) as lat,
ST_Distance(geometry::geography, ST_GeogFromText('SRID=4326;POINT({lon1} {lat1})'))/1000 as distance
from grid
where ST_DWithin(
    geometry::geography, 
    ST_GeogFromText('SRID=4326;POINT({lon1} {lat1})'), 
    {radiusKm*1000} )
order by ST_Distance(geometry, ST_GeomFromText('POINT({lon1} {lat1})',4326));
"""
print(s2)

select sr, ST_Y(geometry) as lon, ST_X(geometry) as lat,
ST_Distance(geometry::geography, ST_GeogFromText('SRID=4326;POINT(73.8534 18.491)'))/1000 as distance
from grid
where ST_DWithin(
    geometry::geography, 
    ST_GeogFromText('SRID=4326;POINT(73.8534 18.491)'), 
    35000 )
order by ST_Distance(geometry, ST_GeomFromText('POINT(73.8534 18.491)',4326));



In [13]:
df2 = pd.read_sql(s2, con=engine)
df2

,sr,lon,lat,distance
0,4324,18.50,73.75,10.964929
1,4293,18.50,74.00,15.513747
2,2185,18.25,73.75,28.826468
3,898,18.75,73.75,30.674459
4,129,18.25,74.00,30.847787
5,3251,18.75,74.00,32.575967
